In [ ]:
from dataclasses import dataclass
from typing import List


@dataclass(frozen=True)
class GameOf24State:
    # game of 24 puzzle, for example 1 1 4 6
    puzzle: str

    # initialized to the same value as puzzle, but is updated as the game progresses
    current_state: str

    steps: List[str]

    #Randomness used for resampling (random seed)
    randomness: int

    def __hash__(self):
        return hash((self.puzzle, self.current_state, " -> ".join(self.steps)))
    
    def items(self):
        return self.puzzle, self.current_state, self.steps, self.randomness
    
    def duplicate(self, randomness=None):
        return GameOf24State(
            puzzle=self.puzzle,
            current_state=self.current_state,
            steps=self.steps,
            randomness=randomness if randomness is not None else self.randomness)

In [ ]:
#Testing the game of 24

import random

states = []
puzzle = "1 1 4 6"
gameEnv = GameOf24State(puzzle=puzzle, current_state=puzzle, steps=[], randomness=random.randint(0, 1000))

# for step in range(2):
#     print(f"Step {step} : Stepping")

print(gameEnv.items)

print("Stepping...")
gameEnv = GameOf24State(puzzle="1 1 4 6", current_state="1 4 6", steps=gameEnv.steps + ["1 * 1 = 1"], randomness=random.randint(0, 1000))

print(gameEnv.items)

print("Stepping...")


gameEnv = GameOf24State(puzzle="1 1 4 6", current_state="1 24", steps=gameEnv.steps + ["4 * 6 = 24"], randomness=random.randint(0, 1000))

print(gameEnv.items)

<bound method GameOf24State.items of GameOf24State(puzzle='1 1 4 6', current_state='1 1 4 6', steps=[], randomness=203)>
Stepping...
<bound method GameOf24State.items of GameOf24State(puzzle='1 1 4 6', current_state='1 4 6', steps=['1 * 1 = 1'], randomness=631)>
Stepping...
<bound method GameOf24State.items of GameOf24State(puzzle='1 1 4 6', current_state='1 24', steps=['1 * 1 = 1', '4 * 6 = 24'], randomness=34)>


In [ ]:
#Reflexion agent :O

import asyncio
import re
import math
import random
import numpy as np
from sympy import simplify

random.seed(0)

from src.prompts.totor import gameof24 as totor_prompts
from prompts.adapt import gameof24 as llama_prompts
from states.gameof24 import GameOf24State
from utils import parse_suggestions, create_box


class GameOf24Agent:

    @staticmethod
    async def step(state: GameOf24State, api, namespace)-> GameOf24State:
        """
        Given a state, returns the next state one.
        """

        # set up the prompt, based on the current state

        # ToT uses bfs_prompt to generate next steps but then uses
        # the cot_prompt to get the final expression. 
        # For example, input : 1 1 4 6
        # Step 0 : '1 - 1 = 0 (left: 0 4 6)'          BFS prompt
        # Step 1 : '0 + 4 = 4 (left: 4 6)'            BFS prompt
        # Step 2 : '4 * 6 = 24 (left: 24)'            BFS prompt
        # Step 3 : Answer : ((1 - 1) + 4) * 6 = 24    CoT prompt


        # set up the prompt, based on the current state
        current_state = state.current_state

        if current_state.strip() == "24":
            # CoT prompt
            steps = "\n".join(state.steps) + "\n"

            # Set up CoT prompt
            if any(author in api.model for author in ["meta", "google", "mistral", "gpt-4o"]):
                prompt = llama_prompts.cot_prompt.format(input=state.puzzle) + "Steps:\n" + steps + "Answer: "
            else:
                prompt = totor_prompts.cot_prompt.format(input=state.puzzle) + "Steps:\n" + steps

            # Get the final expression
            suggestions = await api.buffered_request(prompt, key=hash(state), namespace=namespace)

            # State does not change, only the steps
            selected_suggestion = suggestions
            selected_state = state.current_state
        else:
            # Set up BFS prompt
            if any(author in api.model for author in ["meta", "google", "mistral", "gpt-4o"]):
                prompt = llama_prompts.bfs_prompt.format(input=current_state)
            else:
                prompt = totor_prompts.bfs_prompt.format(input=current_state)

            # Get the next state
            suggestions = await api.buffered_request(prompt, key=hash(state), namespace=namespace)

            # parse suggestions, based on the current state
            parsed_suggestions = parse_suggestions(suggestions)
            if parsed_suggestions == []:
                print(f"No suggestions were paresed from state: {state}")
                print(f"\nPrompt: {prompt}\nSuggestions: {suggestions}\nParsed suggestions: {' | '.join(parsed_suggestions)}\n")
                assert False, "No suggestions found."
            
            suggestions = parsed_suggestions
            
            random.seed(state.randomness)
            selected_suggestion = random.choice(suggestions)
            selected_state = GameOf24Agent.parse_next_state(selected_suggestion)

        # set up new state object
        next_state = GameOf24State(
            puzzle=state.puzzle,
            current_state=selected_state,
            steps=state.steps + [selected_suggestion],
            randomness=random.randint(0, 1000)
        )
        return next_state



ModuleNotFoundError: No module named 'gameof24'